In [1]:
import os
from qgis.core import *
from qgis.utils import *
from PyQt4.QtCore import QFileInfo, QSettings

ImportError: No module named qgis.core

In [ ]:
# supply path to qgis install location
# this will only work if we're on the right environment
QgsApplication.setPrefixPath("/usr/bin/qgis", True)

# create a reference to the QgsApplication, setting the
# second argument to False disables the GUI
qgs = QgsApplication([], False)

# load providers
qgs.initQgis()

In [4]:
! ls ../TrainingData-Trees/

Kolovai-Trees-20180108.cpg  Kolovai-Trees-20180108.qpj
Kolovai-Trees-20180108.dbf  Kolovai-Trees-20180108.shp
Kolovai-Trees-20180108.prj  Kolovai-Trees-20180108.shx


In [ ]:
def clipTrees(rasterPath, vectorPath, searchString, treeType):
    
    # save the aerial image as a raster layer
    rlayer = QgsRasterLayer(rasterPath, 'title: ' + rasterPath)
    if not rlayer.isValid():
        print "rlayer failed to load!"
    else:
        # store the coordinate reference system
        rcrs = rlayer.crs()

    # save the shapefile as a vector layer
    vlayer = QgsVectorLayer(vectorPath, 'title: ' + vectorPath, 'ogr')
    if not vlayer.isValid():
        print "vlayer failed to load!"
    else:
        # store the coordinate reference system
        vcrs = vlayer.crs()
    
    # create an object that converts one coordinate reference system (CRS) to another
    # from the shapefile's CRS to the image's CRS
    xform = QgsCoordinateTransform(vlayer.crs(), rlayer.crs())

    # approximately 8x8 snippet centered on the tree
    xOffset = 4.2854
    yOffset = 4.3105
    
    # create a SQL based feature request object to give us features containing searchString
    exp = QgsExpression('other_tags ILIKE \'%' + searchString + '%\'')
    request = QgsFeatureRequest(exp)

    i = 0
    # iterate through all features matching our request
    for feat in vlayer.getFeatures(request):
        # get geometry of each feature
        geom = feat.geometry()
        
        # if it's a point, use the coordinates to make an 8x8 box centered on the point
        if geom.type() == QGis.Point:
            bbox = xform.transform(feat.geometry().boundingBox())
            # make sure the existing bounding box is not actually a box, but a point
            if (bbox.xMinimum() == bbox.xMaximum()) and (bbox.yMinimum() == bbox.yMaximum()):
                x = bbox.xMinimum()
                y = bbox.yMinimum()
                ulx = x - xOffset
                lry = y - yOffset
                lrx = x + xOffset
                uly = y + yOffset
                
                # shell command that performs the actual clip
                ! gdal_translate -of JPEG -projwin {ulx} {uly} {lrx} {lry}, tonga.tif tree{str(i)}.jpg
                
                # tidy the output into folders
                ! mv tree{str(i)}.jpg automated-clips/{treeType}/
                
                # keep the georeferenced data for future use (outside the scope of the project at this time)
                ! mv tree{str(i)}.jpg.aux.xml automated-clips/{treeType}/aux-xml/
                
                i += 1
                if (treeType == 'cocos') and (i == 3000):
                    # clipping all 10315 coconut trees is gonna take way too long
                    break
            else:
                # this is never expected to occur but just in case
                print "might not be a point"

    print "total trees clipped: ", i

In [ ]:
# Cut out all papaya trees (89)
# clipTrees('tonga.tif', '../TrainingData-Trees/Kolovai-Trees-20180108.shp', 'apaya', 'papaya')

In [ ]:
# Cut out all mango trees (261)
# clipTrees('tonga.tif', '../TrainingData-Trees/Kolovai-Trees-20180108.shp', 'ang', 'mango')

In [ ]:
# # WARNING THIS IS GONNA TAKE FOREVER
# Cut out all banana trees (~2700)
# clipTrees('tonga.tif', '../TrainingData-Trees/Kolovai-Trees-20180108.shp', 'usa', 'musa')

In [ ]:
# # WARNING THIS IS GONNA TAKE FOREVER
# Cut out all coconut trees (~10300)
# clipTrees('tonga.tif', '../TrainingData-Trees/Kolovai-Trees-20180108.shp', 'coc', 'cocos')

In [ ]:
# When complete, call exitQgis() to remove the provider and layer registries from memory
qgs.exitQgis()